## Import Library

In [1]:
import pandas as pd
import torch
import transformers
import wandb
import pprint
from rouge import Rouge

import torch
import time
from trl import SFTTrainer

from transformers import PreTrainedTokenizerFast, T5ForConditionalGeneration, T5Config
from transformers import T5ForConditionalGeneration, T5Tokenizer

from datasets import Dataset, DatasetDict, load_dataset

c:\Users\kimdo\바탕 화면\AI Lab\nlp-competition\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
device

device(type='cuda', index=0)

## Data Preprocessing

In [4]:
data_path = r"C:\Users\kimdo\바탕 화면\AI Lab\nlp-competition\data"
train_df = pd.read_csv(data_path + '/train.csv')
dev_df = pd.read_csv(data_path + '/dev.csv')
test_df = pd.read_csv(data_path + '/test.csv')
submission = pd.read_csv(data_path + '/sample_submission.csv', index_col="Unnamed: 0")

In [5]:
# DataFrame을 Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

# DatasetDict 생성
dataset = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

In [7]:
# 데이터 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('eenzeenee/t5-base-korean-summarization')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
train_data = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

train_df.columns

Index(['fname', 'dialogue', 'summary', 'topic'], dtype='object')

In [10]:
pprint.pprint(train_data[0]["dialogue"])

('#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n'
 '#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n'
 '#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n'
 '#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n'
 '#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 '
 '번은 오세요.\n'
 '#Person2#: 알겠습니다.\n'
 '#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n'
 '#Person2#: 네.\n'
 '#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n'
 '#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n'
 '#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n'
 '#Person2#: 알겠습니다, 감사합니다, 의사선생님.')


## Model Fine-tuning

In [11]:
# 모델 로드
model_id = "eenzeenee/t5-base-korean-summarization"

In [13]:
config_file_path = "C:/Users/kimdo/바탕 화면/AI Lab/nlp-competition/models/eenzeenee/t5-base-korean-summarization/config.json"

# 설정 파일을 통해 T5Config 객체 초기화
t5_config = T5Config.from_pretrained(config_file_path)

# 초기화된 객체 확인
t5_config

T5Config {
  "_name_or_path": "eenzeenee/t5-base-korean-summarization",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 128,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.41.0",
  "use_cache": true,
  "vocab_size": 50358
}

In [14]:
#T5 모델 설정
model = T5ForConditionalGeneration.from_pretrained(model_id,
                                             config=t5_config,
                                             device_map=device
                                             )

In [15]:
model.gradient_checkpointing_enable()

In [16]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [18]:
print_trainable_parameters(model)

trainable params: 275579136 || all params: 275579136 || trainable%: 100.0


In [19]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [20]:
train_data

Dataset({
    features: ['fname', 'dialogue', 'summary', 'topic'],
    num_rows: 12457
})

In [21]:
def make_prompt(example):
    return f"""<s>### Instruction:
다음 대화을 요약해주세요.

### Dialogue:
{example['dialogue']}

### Summary:
{example['summary']} </s>"""


In [22]:
def generate_and_tokenize_prompt(example):
    full_prompt = make_prompt(example)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True,)
    return tokenized_full_prompt

In [23]:
generate_and_tokenize_prompt(train_data[0])

{'input_ids': [29, 84, 31, 4, 4, 4, 222, 42, 79, 5340, 83, 86, 68, 30814, 27, 200, 1021, 222, 2553, 291, 222, 7318, 21899, 15, 200, 200, 4, 4, 4, 222, 37, 74, 6471, 80, 72, 8852, 27, 200, 4, 49, 4253, 43768, 18, 4, 27, 222, 1381, 963, 13, 222, 14563, 796, 15, 222, 425, 274, 222, 528, 21693, 222, 2183, 535, 15, 222, 805, 222, 997, 222, 29297, 296, 32, 200, 4, 49, 4253, 43768, 19, 4, 27, 222, 1323, 6831, 291, 222, 1760, 222, 398, 262, 222, 2150, 222, 398, 222, 2585, 296, 15, 200, 4, 49, 4253, 43768, 18, 4, 27, 222, 5367, 296, 13, 222, 1837, 311, 222, 22, 482, 222, 1063, 222, 1323, 6831, 291, 222, 3629, 222, 6136, 15, 222, 4376, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [26]:
train_data[0]

{'fname': 'train_0',
 'dialogue': '#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.',
 'summary': '스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.',
 'topic': '건강검진 받기'}

In [27]:
#Wandb 연동
wandb.init(
    project='Upstage_AI_Lab_2_nlp',
    entity='kimdohoo1102',
    name = f"eenzeenee_t5-{str(int(time.time()))}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kimdohoo1102. Use `wandb login --relogin` to force relogin


In [28]:
valid_dataset

Dataset({
    features: ['fname', 'dialogue', 'summary', 'topic'],
    num_rows: 499
})

In [29]:
# 모델 성능에 대한 평가 지표를 정의(본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가)
def compute_metrics(config, tokenizer, pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    
    return results

In [31]:
training_args = transformers.TrainingArguments(
    output_dir="outputs",
    
    num_train_epochs = 20,
    
    per_device_train_batch_size=50,
    
    gradient_accumulation_steps=2,
    
    weight_decay = 0.01,
    seed = 42,

    logging_steps=100,
    max_steps=3000,
    
    save_strategy="epoch",
    
    #  max_steps=50,
    learning_rate=1e-5,
    
    fp16=True,
    
    # bf16=True,
    # fp16=False,
    # tf32=True,
      
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    report_to="wandb",
    
)

In [32]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    max_seq_length=256,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=make_prompt, 
    args=training_args,
    compute_metrics = compute_metrics,
)


max_steps is given, it will override any value given in num_train_epochs
c:\Users\kimdo\바탕 화면\AI Lab\nlp-competition\.venv\Lib\site-packages\trl\trainer\sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


## Train

In [33]:
import gc

In [34]:
gc.collect()

560

In [35]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}
  3%|▎         | 100/3000 [05:01<2:21:26,  2.93s/it]

{'loss': 5.4009, 'grad_norm': 1.5353789329528809, 'learning_rate': 9.999708626830617e-06, 'epoch': 0.49}


  7%|▋         | 200/3000 [09:52<2:13:43,  2.87s/it]

{'loss': 0.0712, 'grad_norm': 0.14838486909866333, 'learning_rate': 9.964784918620284e-06, 'epoch': 0.98}


  7%|▋         | 204/3000 [10:04<2:14:20,  2.88s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}
 10%|█         | 300/3000 [14:49<2:12:31,  2.95s/it]

...

100%|██████████| 3000/3000 [2:27:32<00:00,  2.94s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}


{'loss': 0.0012, 'grad_norm': 0.014188890345394611, 'learning_rate': 0.0, 'epoch': 14.67}


100%|██████████| 3000/3000 [2:27:33<00:00,  2.95s/it]

{'train_runtime': 8856.9086, 'train_samples_per_second': 33.872, 'train_steps_per_second': 0.339, 'train_loss': 0.1864653771718343, 'epoch': 14.67}


TrainOutput(global_step=3000, training_loss=0.1864653771718343, metrics={'train_runtime': 8856.9086, 'train_samples_per_second': 33.872, 'train_steps_per_second': 0.339, 'total_flos': 1.0898679478616064e+17, 'train_loss': 0.1864653771718343, 'epoch': 14.669926650366747})

In [36]:
trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}


In [37]:
# WandB 함수 호출 및 알림
wandb.alert(title="train end", text="모델 학습종료", level="WARN")

## Load Model & Inferrence

In [69]:
device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')

In [70]:
model_folder = r"C:\Users\kimdo\바탕 화면\AI Lab\nlp-competition\code\outputs\checkpoint-3000"

In [71]:
import torch
from tqdm import tqdm 
import wandb
from rouge import Rouge

import torch
import time

from transformers import PreTrainedTokenizerFast, T5ForConditionalGeneration

In [72]:
# 모델과 토크나이저 불러오기
model = T5ForConditionalGeneration.from_pretrained(
    model_folder,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map=device,
)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_folder)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [73]:
def make_prompt_for_test(example):
    return f"""<s>### Instruction:
다음 대화를 요약해주세요.

### Dialogue:
{example}

### Summary:
"""


In [74]:
test_dataset

Dataset({
    features: ['fname', 'dialogue'],
    num_rows: 499
})

In [75]:
len(test_dataset)

499

In [76]:
test_dataset[0]["dialogue"]

'#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, 실장님...\n#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?\n#Person2#: 네, 실장님. 시작하셔도 됩니다.\n#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.\n#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?\n#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.\n#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.\n#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?\n#Person2#: 이것은 내부와 외부 통신에 적용됩니다.\n#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.\n#Person2#: 그게 다신가요?\n#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.'

In [ ]:
sample = valid_dataset[10]

prompt = f"""### Instruction:
다음 대화을 요약해주세요.

### Dialogue:
{sample['dialogue']}

### Summary:
"""
print(prompt)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=50, temperature=0.7)

print('Output:\n',
      tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):])
print('\nGround truth:\n', sample['summary'])

(오류 발생부분으로 수정예정)

In [ ]:
import torch
from tqdm import tqdm

# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델을 평가 모드로 설정
model.eval()

for idx, dialogue in enumerate(tqdm(test_dataset['dialogue'])):
    prompt = make_prompt_for_test(dialogue)
    
    # 입력을 토크나이징
    inputs = tokenizer(prompt, padding=True, truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs["input_ids"]
    
    # GPU가 사용 가능한 경우 텐서를 GPU로 이동
    input_ids = input_ids.to(device)

    # 모델에 텐서를 직접 전달하여 출력 생성
    outputs = model.generate(input_ids=input_ids)

    # 생성된 텍스트 디코딩
    summary = tokenizer.decode(outputs[0][0], skip_special_tokens=True)
    print(f"Summary for idx {idx}: {summary}")
    
    # 결과를 submission DataFrame에 저장
    submission.loc[idx, 'summary'] = summary

In [ ]:
submission

In [66]:
# Save the submission file
submission.to_csv(f"submission_eenzt5_{str(int(time.time()))}.csv", index=False)

In [63]:
wandb.finish()

train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/grad_norm,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,█████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,1.0898679478616064e+17
train/epoch,14.66993
train/global_step,3000
train/grad_norm,0.01419
train/learning_rate,0.0
train/loss,0.0012
